In [2]:
import pandas as pd
import numpy as np
import json
import ast
import datetime
from tqdm.notebook import tqdm
import pandasql as ps

In [3]:
# The MyData/StreamingHistory{k}.json is the streaming history provided by the spotify client
# The data is retrieved by going to your spotify account on Spotify.com:
## https://www.spotify.com/dk-da/account/privacy/file_in = 'StreamingHistory3_enriched.csv'
# The entiched data is obtained from the notebook: 1_spotify_enrichment.ipynb
file_in = 'StreamingHistory3_enriched.csv'
# Determine the name of the enriched csv-file we will end up using this notebook
file_out = 'StreamingHistory3_enriched_activity.csv'

In [4]:
df = pd.read_csv(file_in)

In [5]:
df['dayofweek_nr'] = df.endTime.apply(lambda x: pd.to_datetime(x).dayofweek)

In [6]:
df

,endTime,track_id,track_name,track_artist,genres,msPlayed,audio_data,danceability,energy,key,...,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,startTime,dayofweek_nr
0,2021-12-30 19:06:00,0k1WUmIRnG3xU6fvvDVfRG,Way 2 Sexy (with Future & Young Thug),Drake,"['canadian hip hop', 'canadian pop', 'hip hop'...",38165.0,"[{'danceability': 0.803, 'energy': 0.597, 'key...",0.803,0.597,11,...,136.008,audio_features,0k1WUmIRnG3xU6fvvDVfRG,spotify:track:0k1WUmIRnG3xU6fvvDVfRG,https://api.spotify.com/v1/tracks/0k1WUmIRnG3x...,https://api.spotify.com/v1/audio-analysis/0k1W...,257605,4,2021-12-30 19:05:21.835,3
1,2021-12-30 19:06:00,4eL3XeuGaEoVT8ttDh3hwY,too easy,Gunna,"['atl hip hop', 'melodic rap', 'rap', 'trap']",4224.0,"[{'danceability': 0.798, 'energy': 0.574, 'key...",0.798,0.574,1,...,155.964,audio_features,4eL3XeuGaEoVT8ttDh3hwY,spotify:track:4eL3XeuGaEoVT8ttDh3hwY,https://api.spotify.com/v1/tracks/4eL3XeuGaEoV...,https://api.spotify.com/v1/audio-analysis/4eL3...,138587,4,2021-12-30 19:05:55.776,3
2,2021-12-30 19:06:00,5yY9lUy8nbvjM1Uyo1Uqoc,Life Is Good (feat. Drake),Future,"['atl hip hop', 'hip hop', 'pop rap', 'rap', '...",6506.0,"[{'danceability': 0.676, 'energy': 0.609, 'key...",0.676,0.609,2,...,142.037,audio_features,5yY9lUy8nbvjM1Uyo1Uqoc,spotify:track:5yY9lUy8nbvjM1Uyo1Uqoc,https://api.spotify.com/v1/tracks/5yY9lUy8nbvj...,https://api.spotify.com/v1/audio-analysis/5yY9...,237735,4,2021-12-30 19:05:53.494,3
3,2021-12-30 19:07:00,0k1WUmIRnG3xU6fvvDVfRG,Way 2 Sexy (with Future & Young Thug),Drake,"['canadian hip hop', 'canadian pop', 'hip hop'...",15935.0,"[{'danceability': 0.803, 'energy': 0.597, 'key...",0.803,0.597,11,...,136.008,audio_features,0k1WUmIRnG3xU6fvvDVfRG,spotify:track:0k1WUmIRnG3xU6fvvDVfRG,https://api.spotify.com/v1/tracks/0k1WUmIRnG3x...,https://api.spotify.com/v1/audio-analysis/0k1W...,257605,4,2021-12-30 19:06:44.065,3
4,2021-12-30 19:07:00,0k7wmahjkn389wAZdz19Cv,Drankin N Smokin,Future,"['atl hip hop', 'hip hop', 'pop rap', 'rap', '...",59520.0,"[{'danceability': 0.752, 'energy': 0.658, 'key...",0.752,0.658,0,...,151.951,audio_features,0k7wmahjkn389wAZdz19Cv,spotify:track:0k7wmahjkn389wAZdz19Cv,https://api.spotify.com/v1/tracks/0k7wmahjkn38...,https://api.spotify.com/v1/audio-analysis/0k7w...,213520,4,2021-12-30 19:06:00.480,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8957,2022-04-01 19:07:00,49QK41xxLFFPVLm3caInNB,Santé,Stromae,"['belgian pop', 'g-house']",190999.0,"[{'danceability': 0.795, 'energy': 0.67, 'key'...",0.795,0.670,0,...,137.920,audio_features,49QK41xxLFFPVLm3caInNB,spotify:track:49QK41xxLFFPVLm3caInNB,https://api.spotify.com/v1/tracks/49QK41xxLFFP...,https://api.spotify.com/v1/audio-analysis/49QK...,191000,3,2022-04-01 19:03:49.001,4
8958,2022-04-01 19:10:00,58HvfVOeJY7lUuCqF0m3ly,MIDDLE OF THE NIGHT,Elley Duhé,"['alt z', 'pop']",184447.0,"[{'danceability': 0.41, 'energy': 0.611, 'key'...",0.410,0.611,4,...,185.727,audio_features,58HvfVOeJY7lUuCqF0m3ly,spotify:track:58HvfVOeJY7lUuCqF0m3ly,https://api.spotify.com/v1/tracks/58HvfVOeJY7l...,https://api.spotify.com/v1/audio-analysis/58Hv...,184448,3,2022-04-01 19:06:55.553,4
8959,2022-04-01 19:13:00,3JiVOU9tXbRi7aE8hBaz5v,Kopenhagen (feat. ICEKIID & Noah Carter),Gilli,"['danish hip hop', 'danish pop']",147426.0,"[{'danceability': 0.747, 'energy': 0.672, 'key...",0.747,0.672,11,...,99.719,audio_features,3JiVOU9tXbRi7aE8hBaz5v,spotify:track:3JiVOU9tXbRi7aE8hBaz5v,https://api.spotify.com/v1/tracks/3JiVOU9tXbRi...,https://api.spotify.com/v1/audio-analysis/3JiV...,147427,4,2022-04-01 19:10:32.574,4
8960,2022-04-01 19:18:00,0lVcRfuCL9U6DCmyAJAqmV,Tag mig tilbage,Nik & Jay,"['danish hip hop', 'danish pop']",292665.0,"[{'danceability': 0.776, 'energy': 0.893, 'key...",0.776,0.893,6,...,89.021,audio_features,0lVcRfuCL9U6DCmyAJAqmV,spotify:track:0lVcRfuCL9U6DCmyAJAqmV,https://api.spotify.com/v1/tracks/0lVcRfuCL9U6...,https://api.spotify.com/v1/audio-analysis/0lVc...,292667,4,2022-04-01 1

In [7]:
df['day_name'] = df.endTime.apply(lambda x: pd.to_datetime(x).day_name())
df['msPlayed'] = df['msPlayed'].astype(float)
df['endTime'] = pd.to_datetime(df['endTime'])
#df['startTime'] = 0
# Figuring out when the song was started (quite trivial)
for idx, row in tqdm(enumerate(df.values)):
    res = pd.to_datetime(df.iloc[idx].endTime + datetime.timedelta(milliseconds=-df.iloc[idx]['msPlayed']))
    df.loc[idx, "startTime"] = res

In [8]:
# Basic formatting
df['startTime_hour'] = pd.to_datetime(df['startTime']).dt.hour

In [9]:
df['month_nr'] = df.endTime.apply(lambda x: pd.to_datetime(x).month).astype(int)
df['month_name'] = df.endTime.apply(lambda x: pd.to_datetime(x).month_name())

#df

In [10]:
#df[['month_nr', 'month_name']].sort_values('month_nr')

In [11]:
# Work
# Study
# Exercise


In [12]:
# Custom function to apply what activity I was doing when
# Adding this to the dataframe as column "activity type"
def activity_type(row):
    month_start = 9
    month_end = 12
    # September - December
    # Monday
    if month_start <= row['month_nr'] <= month_end and row['day_name'] == 'Monday' and 8 <= row['startTime_hour'] <= 12:
        return 'study'
    if month_start <= row['month_nr'] <= month_end and row['day_name'] == 'Monday' and 13 <= row['startTime_hour'] <= 17:
        return 'work'
    if month_start <= row['month_nr'] <= month_end and row['day_name'] == 'Monday' and 18 <= row['startTime_hour'] <= 20:
        return 'exercise'
    
    # Tuesday
    if month_start <= row['month_nr'] <= month_end and row['day_name'] == 'Tuesday' and 10 <= row['startTime_hour'] <= 17:
        return 'work'
    if month_start <= row['month_nr'] <= month_end and row['day_name'] == 'Tuesday' and 17 <= row['startTime_hour'] <= 20:
        return 'study'
    
    # Wednesday
    if month_start <= row['month_nr'] <= month_end and row['day_name'] == 'Wednesday' and (8 <= row['startTime_hour'] <= 12 or 13 <= row['startTime_hour'] <= 17):
        return 'study'
    if month_start <= row['month_nr'] <= month_end and row['day_name'] == 'Wednesday' and 18 <= row['startTime_hour'] <= 21:
        return 'exercise'
    
    # Thursday
    if month_start <= row['month_nr'] <= month_end and row['day_name'] == 'Thursday' and 10 <= row['startTime_hour'] <= 17:
        return 'work'
    # Thursday
    if month_start <= row['month_nr'] <= month_end and row['day_name'] == 'Thursday' and 17 <= row['startTime_hour'] <= 20:
        return 'study'
    
    # Friday
    if month_start <= row['month_nr'] <= month_end and row['day_name'] == 'Friday' and 8 <= row['startTime_hour'] <= 12:
        return 'study'

    # Saturday
    if month_start <= row['month_nr'] <= month_end and row['day_name'] == 'Saturday' and 10 <= row['startTime_hour'] <= 12:
        return 'exercise'
    if month_start <= row['month_nr'] <= month_end and row['day_name'] == 'Saturday':
        return 'study'
    
    # Sunday
    if month_start <= row['month_nr'] <= month_end and row['day_name'] == 'Sunday' and 10 <= row['startTime_hour'] <= 12:
        return 'exercise'
    if month_start <= row['month_nr'] <= month_end and row['day_name'] == 'Sunday' and 13 <= row['startTime_hour'] <= 16:
        return 'study'

    
    month_start = 1
    month_end = 6
    # January - June
    # Monday
    if month_start <= row['month_nr'] <= month_end and row['day_name'] == 'Monday' and 8 <= row['startTime_hour'] <= 12:
        return 'study'
    if month_start <= row['month_nr'] <= month_end and row['day_name'] == 'Monday' and 13 <= row['startTime_hour'] <= 17:
        return 'work'
    if month_start <= row['month_nr'] <= month_end and row['day_name'] == 'Monday' and 18 <= row['startTime_hour'] <= 20:
        return 'exercise'
    
    # Tuesday
    if month_start <= row['month_nr'] <= month_end and row['day_name'] == 'Tuesday' and 10 <= row['startTime_hour'] <= 17:
        return 'work'
    if month_start <= row['month_nr'] <= month_end and row['day_name'] == 'Tuesday' and 17 <= row['startTime_hour'] <= 20:
        return 'study'
    
    # Wednesday
    if month_start <= row['month_nr'] <= month_end and row['day_name'] == 'Wednesday' and (8 <= row['startTime_hour'] <= 12 or 13 <= row['startTime_hour'] <= 17):
        return 'study'
    if month_start <= row['month_nr'] <= month_end and row['day_name'] == 'Wednesday' and 18 <= row['startTime_hour'] <= 21:
        return 'exercise'
    
    # Thursday
    if month_start <= row['month_nr'] <= month_end and row['day_name'] == 'Thursday' and 10 <= row['startTime_hour'] <= 17:
        return 'work'
    # Thursday
    if month_start <= row['month_nr'] <= month_end and row['day_name'] == 'Thursday' and 17 <= row['startTime_hour'] <= 20:
        return 'study'
    
    # Friday
    if month_start <= row['month_nr'] <= month_end and row['day_name'] == 'Friday' and 8 <= row['startTime_hour'] <= 12:
        return 'study'

    # Saturday
    if month_start <= row['month_nr'] <= month_end and row['day_name'] == 'Saturday' and 10 <= row['startTime_hour'] <= 12:
        return 'exercise'
    if month_start <= row['month_nr'] <= month_end and row['day_name'] == 'Saturday':
        return 'study'
    
    # Sunday
    if month_start <= row['month_nr'] <= month_end and row['day_name'] == 'Sunday' and 10 <= row['startTime_hour'] <= 12:
        return 'exercise'
    if month_start <= row['month_nr'] <= month_end and row['day_name'] == 'Sunday' and 13 <= row['startTime_hour'] <= 16:
        return 'study'
    
    else:
        return 'relax'

In [13]:
df['activity_type'] = df.apply(lambda row: activity_type(row), axis=1)

In [14]:
df.head(5)

,endTime,track_id,track_name,track_artist,genres,msPlayed,audio_data,danceability,energy,key,...,analysis_url,duration_ms,time_signature,startTime,dayofweek_nr,day_name,startTime_hour,month_nr,month_name,activity_type
0,2021-12-30 19:06:00,0k1WUmIRnG3xU6fvvDVfRG,Way 2 Sexy (with Future & Young Thug),Drake,"['canadian hip hop', 'canadian pop', 'hip hop'...",38165.0,"[{'danceability': 0.803, 'energy': 0.597, 'key...",0.803,0.597,11,...,https://api.spotify.com/v1/audio-analysis/0k1W...,257605,4,2021-12-30 19:05:21.835000,3,Thursday,19,12,December,study
1,2021-12-30 19:06:00,4eL3XeuGaEoVT8ttDh3hwY,too easy,Gunna,"['atl hip hop', 'melodic rap', 'rap', 'trap']",4224.0,"[{'danceability': 0.798, 'energy': 0.574, 'key...",0.798,0.574,1,...,https://api.spotify.com/v1/audio-analysis/4eL3...,138587,4,2021-12-30 19:05:55.776000,3,Thursday,19,12,December,study
2,2021-12-30 19:06:00,5yY9lUy8nbvjM1Uyo1Uqoc,Life Is Good (feat. Drake),Future,"['atl hip hop', 'hip hop', 'pop rap', 'rap', '...",6506.0,"[{'danceability': 0.676, 'energy': 0.609, 'key...",0.676,0.609,2,...,https://api.spotify.com/v1/audio-analysis/5yY9...,237735,4,2021-12-30 19:05:53.494000,3,Thursday,19,12,December,study
3,2021-12-30 19:07:00,0k1WUmIRnG3xU6fvvDVfRG,Way 2 Sexy (with Future & Young Thug),Drake,"['canadian hip hop', 'canadian pop', 'hip hop'...",15935.0,"[{'danceability': 0.803, 'energy': 0.597, 'key...",0.803,0.597,11,...,https://api.spotify.com/v1/audio-analysis/0k1W...,257605,4,2021-12-30 19:06:44.065000,3,Thursday,19,12,December,study
4,2021-12-30 19:07:00,0k7wmahjkn389wAZdz19Cv,Drankin N Smokin,Future,"['atl hip hop', 'hip hop', 'pop rap', 'rap', '...",59520.0,"[{'danceability': 0.752, 'energy': 0.658, 'key...",0.752,0.658,0,...,https://api.spotify.com/v1/audio-analysis/0k7w...,213520,4,2021-12-30 19:06:00.480000,3,Thursday,19,12,December,study


In [15]:
df.activity_type.value_counts()

study       3758
relax       2289
work        2257
exercise     658
Name: activity_type, dtype: int64

In [16]:
#import random

In [17]:
#import random
#def assign_productivity_level(x):
#    productivity_levels = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
#    weights = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
#    
#    return random.choices(productivity_levels, 
#                          weights=weights, 
#                          k=1
#                         )[0]
#    

In [18]:
#df['productivity_level'] = df.apply(lambda x: assign_productivity_level(x), axis=1)

In [19]:
df.to_csv(file_out, index=False)

NameError: name 'df' is not defined